In [4]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
import os
import sys

root_dir = os.path.abspath(os.path.join(os.getcwd(), '../..'))
print(root_dir)
assert os.path.exists(root_dir), f'Could not find root directory at {root_dir}'
sys.path.insert(0, root_dir)

from custom_utils.config_handler import read_config, pprint_config

/Users/user010/Desktop/Programming/ML/En2RuTranslator


In [6]:
overrides = ["setup=inference"]
cfg = read_config(overrides=overrides)
pprint_config(cfg)

{
  "root": "/Users/user010/Desktop/Programming/ML/En2RuTranslator",
  "opus_model": {
    "name": "opus-en-ru",
    "model_and_tokenizer_name": "Helsinki-NLP/opus-mt-en-ru"
  },
  "nnlb_model": {
    "name": "nnlb-1.3B-distilled",
    "model_and_tokenizer_name": "facebook/nllb-200-distilled-1.3B"
  },
  "inference_dataset": {
    "path": "/Users/user010/Desktop/Programming/ML/En2RuTranslator/data/processed/model_eval.csv"
  },
  "result_dataset": {
    "path": "/Users/user010/Desktop/Programming/ML/En2RuTranslator/data/processed/model_eval_results.csv",
    "cols": {
      "reference": "target",
      "candidates": [
        "transformer-en-ru",
        "nnlb-1.3B-distilled"
      ]
    }
  }
}


In [17]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
sample_texts = ["Hey, how are you?", "My name is John Smith, I live in the United States of America.",
                "I love NLP and Transformers!"]

def get_translations(model: AutoModelForSeq2SeqLM, tokenizer: AutoTokenizer, sample_texts: list,
                     special_gen_params: dict = None) -> list:
    special_gen_params = special_gen_params or {}
    print("Tokenizing...")
    inputs = tokenizer(sample_texts, return_tensors="pt", padding=True, truncation=True, max_length=600)
    print("Generating...")
    translated_tokens = model.generate(
            **inputs,
            **special_gen_params,
            max_length=600,
            early_stopping=True
        )
    print("Decoding...")
    translated_texts = tokenizer.batch_decode(translated_tokens, skip_special_tokens=True)
    return translated_texts

def print_translations(source: list[str], target: list[str]):
    assert len(source) == len(target), "Source and target lists must be of same length"
    for src, tgt in zip(source, target):
        print(f"Source: {src}")
        print(f"Target: {tgt}")
        print()

In [19]:
model_and_tokenizer_name = cfg.nnlb_model.model_and_tokenizer_name
nnlb_tokenizer = AutoTokenizer.from_pretrained(model_and_tokenizer_name)
nnlb_model = AutoModelForSeq2SeqLM.from_pretrained(model_and_tokenizer_name)

In [20]:
translations = get_translations(nnlb_model, nnlb_tokenizer, sample_texts, 
                                special_gen_params={"forced_bos_token_id": nnlb_tokenizer.lang_code_to_id["rus_Cyrl"]})
print_translations(sample_texts, translations)

Tokenizing...
Generating...


/Users/user010/Desktop/Programming/ML/En2RuTranslator/venv/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:399: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(


Decoding...
Source: Hey, how are you?
Target: Привет, как дела?

Source: My name is John Smith, I live in the United States of America.
Target: Меня зовут Джон Смит, я живу в Соединенных Штатах Америки.

Source: I love NLP and Transformers!
Target: Я люблю НЛП и Трансформеров!



In [21]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_and_tokenizer_name = cfg.opus_model.model_and_tokenizer_name
opus_tokenizer = AutoTokenizer.from_pretrained(model_and_tokenizer_name)
opus_model = AutoModelForSeq2SeqLM.from_pretrained(model_and_tokenizer_name)

/Users/user010/Desktop/Programming/ML/En2RuTranslator/venv/lib/python3.11/site-packages/transformers/models/marian/tokenization_marian.py:197: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [22]:
translations = get_translations(opus_model, opus_tokenizer, sample_texts)
print_translations(sample_texts, translations)

Tokenizing...
Generating...
Decoding...
Source: Hey, how are you?
Target: Привет, как дела?

Source: My name is John Smith, I live in the United States of America.
Target: Меня зовут Джон Смит, я живу в Соединенных Штатах Америки.

Source: I love NLP and Transformers!
Target: Я люблю NLP и Transformers!



In [23]:
import pandas as pd
inference_ds = pd.read_csv(cfg.inference_dataset.path)
inference_ds.head()

,source,target
0,The intention would also be to infiltrate terr...,Террористов также обучают методам проникновени...
1,Officials say that as the latest information a...,"По последним данным представителей власти , в ..."
2,While the Balakot camp was reactivated by the ...,Джаиш-е-Мухаммад возобновили работу террористи...
3,The incident in which Pakistan used drones to ...,В качестве яркого примера новой стратегии паки...
4,Officials tell OneIndia that terror groups wou...,Представители власти рассказали порталу OneInd...


In [24]:
from tqdm import tqdm

batch_size = 32
num_batches = len(inference_ds) // batch_size + 1

In [26]:
# nnlb
translations = []
for i in tqdm(range(num_batches)):
    batch = inference_ds.iloc[i*batch_size:(i+1)*batch_size]["source"].tolist()
    batch_translations = get_translations(nnlb_model, nnlb_tokenizer, batch,
                                          special_gen_params={"forced_bos_token_id": nnlb_tokenizer.lang_code_to_id["rus_Cyrl"]})
    translations.extend(batch_translations)

  0%|          | 0/32 [00:00<?, ?it/s]/Users/user010/Desktop/Programming/ML/En2RuTranslator/venv/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:399: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(


Tokenizing...
Generating...


  3%|▎         | 1/32 [00:33<17:07, 33.14s/it]

Decoding...
Tokenizing...
Generating...


  6%|▋         | 2/32 [01:01<15:06, 30.21s/it]

...

Decoding...
Tokenizing...
Generating...


 97%|█████████▋| 31/32 [17:52<00:35, 35.49s/it]

Decoding...
Tokenizing...
Generating...


100%|██████████| 32/32 [18:12<00:00, 34.14s/it]

Decoding...


In [35]:
inference_ds[cfg.nnlb_model.name] = translations

In [28]:
# opus
translations = []
for i in tqdm(range(num_batches)):
    batch = inference_ds.iloc[i*batch_size:(i+1)*batch_size]["source"].tolist()
    batch_translations = get_translations(opus_model, opus_tokenizer, batch)
    translations.extend(batch_translations)

  0%|          | 0/32 [00:00<?, ?it/s]

Tokenizing...
Generating...


  3%|▎         | 1/32 [00:11<05:50, 11.30s/it]

Decoding...
Tokenizing...
Generating...


  6%|▋         | 2/32 [00:22<05:35, 11.17s/it]

...

Decoding...
Tokenizing...
Generating...


 97%|█████████▋| 31/32 [06:21<00:13, 13.85s/it]

Decoding...
Tokenizing...
Generating...


100%|██████████| 32/32 [06:23<00:00, 11.99s/it]

Decoding...


In [29]:
inference_ds[cfg.opus_model.name] = translations

In [36]:
inference_ds.head()

,source,target,transformer-en-ru,nnlb-1.3B-distilled
0,The intention would also be to infiltrate terr...,Террористов также обучают методам проникновени...,"Кроме того, намерение состоит в том, чтобы про...",Имеется в виду также проникновение террористов...
1,Officials say that as the latest information a...,"По последним данным представителей власти , в ...","Официальные лица говорят, что в качестве самой...","Официальные лица говорят, что по последней инф..."
2,While the Balakot camp was reactivated by the ...,Джаиш-е-Мухаммад возобновили работу террористи...,В то время как лагерь в Балакоте был восстанов...,В то время как лагерь Балакота был активирован...
3,The incident in which Pakistan used drones to ...,В качестве яркого примера новой стратегии паки...,"Инцидент, в ходе которого Пакистан использовал...","Инцидент, когда Пакистан использовал беспилотн..."
4,Officials tell OneIndia that terror groups wou...,Представители власти рассказали порталу OneInd...,"Официальные лица сообщают одной Индии, что тер...","Официальные лица говорят OneIndia, что террори..."


In [37]:
inference_ds.to_csv(cfg.result_dataset.path, index=False)